## URL of reference
https://www.kaggle.com/jpmiller/baseline-from-host-data

## code

In [18]:
import pandas as pd
import glob
import numpy as np
from tqdm.notebook import tqdm

### read train / test / submission file 

In [3]:
train_baseline = "/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv"
test_baseline = "/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_test.csv"
submission = "/work/data/input/google-smartphone-decimeter-challenge/sample_submission.csv"

train_df = pd.read_csv(train_baseline)
test_df = pd.read_csv(test_baseline)
sub_df = pd.read_csv(submission)

### merge all truth file

In [20]:
Truth_FILES = glob.glob("/work/data/input/google-smartphone-decimeter-challenge/train/*/*/ground_truth.csv")

cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg','lngDeg']
df_list = [];

df_list = [ pd.read_csv(t,usecols=cols) for t in tqdm(Truth_FILES,total=73) ]
df_truth = pd.concat(df_list,ignore_index=True);

  0%|          | 0/73 [00:00<?, ?it/s]

## generate dataframe containing all truth_path info and baseline prediction info

In [25]:
df_basepread = train_df[cols];
df_all = df_truth.merge(df_basepread,how="inner",on=cols[:3],suffixes=('_truth',"_basepred"));
df_all.head(3)

,collectionName,phoneName,millisSinceGpsEpoch,latDeg_truth,lngDeg_truth,latDeg_basepred,lngDeg_basepred
0,2020-06-04-US-MTV-1,Pixel4,1275339493441,37.416314,-122.080465,37.416345,-122.080528
1,2020-06-04-US-MTV-1,Pixel4,1275339494441,37.416314,-122.080465,37.416355,-122.080497
2,2020-06-04-US-MTV-1,Pixel4,1275339495441,37.416314,-122.080465,37.416346,-122.080499


In [27]:
display(df_all[:6])

,collectionName,phoneName,millisSinceGpsEpoch,latDeg_truth,lngDeg_truth,latDeg_basepred,lngDeg_basepred
0,2020-06-04-US-MTV-1,Pixel4,1275339493441,37.416314,-122.080465,37.416345,-122.080528
1,2020-06-04-US-MTV-1,Pixel4,1275339494441,37.416314,-122.080465,37.416355,-122.080497
2,2020-06-04-US-MTV-1,Pixel4,1275339495441,37.416314,-122.080465,37.416346,-122.080499
3,2020-06-04-US-MTV-1,Pixel4,1275339496441,37.416314,-122.080465,37.416353,-122.080508
4,2020-06-04-US-MTV-1,Pixel4,1275339497441,37.416314,-122.080465,37.416349,-122.080509
5,2020-06-04-US-MTV-1,Pixel4,1275339498441,37.416314,-122.080465,37.416364,-122.080510


In [28]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """
    Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [32]:
df_all["dist"] = calc_haversine(df_all["latDeg_truth"],df_all["lngDeg_truth"],df_all["latDeg_basepred"],df_all["lngDeg_basepred"])

df_all.dist.mean()

3.8468483749951625

In [35]:
display(df_all)

,collectionName,phoneName,millisSinceGpsEpoch,latDeg_truth,lngDeg_truth,latDeg_basepred,lngDeg_basepred,dist
0,2020-06-04-US-MTV-1,Pixel4,1275339493441,37.416314,-122.080465,37.416345,-122.080528,6.588113
1,2020-06-04-US-MTV-1,Pixel4,1275339494441,37.416314,-122.080465,37.416355,-122.080497,5.335392
2,2020-06-04-US-MTV-1,Pixel4,1275339495441,37.416314,-122.080465,37.416346,-122.080499,4.684049
3,2020-06-04-US-MTV-1,Pixel4,1275339496441,37.416314,-122.080465,37.416353,-122.080508,5.751620
4,2020-06-04-US-MTV-1,Pixel4,1275339497441,37.416314,-122.080465,37.416349,-122.080509,5.507901
...,...,...,...,...,...,...,...,...
131337,2020-08-06-US-MTV-2,Mi8,1280791291000,37.427345,-122.069910,37.427365,-122.069899,2.425719
131338,2020-08-06-US-MTV-2,Mi8,1280791292000,37.427345,-122.069910,37.427354,-122.069925,1.623460
131339,2020-08-06-US-MTV-2,Mi8,1280791293000,37.427345,-122.069910,37.427352,-122.069918,1.107135
131340,2020-08-06-US-MTV-2,Mi8,1280791294000,37.427345,-122.069910,37.427366,-122.069897,2.603942


In [36]:
df_all.sort_values(by="dist",ascending=False)[["collectionName","dist"]]

,collectionName,dist
16239,2020-09-04-US-SF-1,8340.257976
17751,2020-07-17-US-MTV-2,5050.995543
109057,2021-04-26-US-SVL-1,2254.344928
16759,2020-07-17-US-MTV-2,2026.294654
16757,2020-07-17-US-MTV-2,1934.676643
...,...,...
113925,2021-01-05-US-SVL-1,0.014395
30148,2021-01-04-US-RWC-1,0.011386
26349,2021-01-04-US-RWC-1,0.011203
55988,2020-09-04-US-SF-2,0.010101
